# ORMs

## What is an ORM?

[Object Relational Mapping](https://en.wikipedia.org/wiki/Object-relational_mapping) _(aka ORM)_ is a technique that lets you query and manipulate data from a database using an object-oriented paradigm. 

An ORM library, written in the language of your choice, encapsulates the codes needed to manipulate the data so you don't use SQL anymore; you interact directly with an object in the same language you're using.

### Pros

#### It saves a lot of time

- [DRY](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself): You write your data model in only one place, and it's easier to update, maintain, and reuse the code.
- A lot of stuff is done automatically, from database handling to I18N.
- It forces you to write MVC code, which, in the end, makes your code a little cleaner.
- You don't have to write poorly-formed SQL (most Web programmers really suck at it, because SQL is treated like a "sub" language, when in reality it's a very powerful and complex one)
- Sanitizing; using prepared statements or transactions are as easy as calling a method.

#### It is more flexible

- It fits in your natural way of coding (it's your language!)
- It abstracts the DB system, so you can change it whenever you want and it won't affect your code.
- The model is weakly bound to the rest of the application, so you can change it or use it anywhere else.
- It lets you use OOP goodness like data inheritance without a headache.

### Con

#### It can be a pain
- You have to learn it, and ORM libraries are not lightweight tools;
- You have to set it up. Same problem.
- Performance is OK for usual queries, but a SQL master will always do better with his own SQL for big projects.
- It abstracts the DB. While it's OK if you know what's happening behind the scene, it's a trap for new programmers that can write very greedy statements, like a heavy hit in a __`for`__ loop.

### SQLAlchemy and Peewee

We'll take a quick look at two Python ORM's and explore some of their features. It's up to you to study them in depth but you'll have to use one in your exercises or projects. We won't cover all of their features and functionalities but the goal is to give you an idea of how they are used.

## 18.1 SQLAlchemy

> SQLAlchemy is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL.

> SQL databases behave less like object collections the more size and performance start to matter; object collections behave less like tables and rows the more abstraction starts to matter. SQLAlchemy aims to accommodate both of these principles.

>SQLAlchemy considers the database to be a relational algebra engine, not just a collection of tables. Rows can be selected from not only tables but also joins and other select statements; any of these units can be composed into a larger structure. SQLAlchemy's expression language builds on this concept from its core.

> The main goal of SQLAlchemy is to change the way you think about databases and SQL!

### We'll cover how to:

- Declare a Mapping
- Configure a Database Engine
- Create a Schema
- Create a Session
- Create, Add and Update Objects
- Rollback Changes
- Query the Session

But first, install SQLAlchemy:

In [ ]:
!pip install SQLAlchemy

### Declaring a Mapping

When using the ORM, we need to describe the database tables we'll be dealing with by defining classes, which will be mapped to those tables. This usually goes into the __`models.py`__ file of your application.

We'll be using a system in SQLAlchemy known as __`Declarative`__, which allows us to create classes that include directives to describe the actual database table they will be mapped to. Outside of what the mapping process does to our class, the class remains otherwise mostly a normal Python class, to which we can define any number of ordinary attributes and methods needed by our application.

- A class using __`Declarative`__ at a minimum needs a __`__tablename__`__ attribute, and at least one Column which is part of a primary key.

In [ ]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()


class User(Base):
        __tablename__ = 'users'

        id = Column(Integer, primary_key=True)
        name = Column(String)
        fullname = Column(String)
        password = Column(String)

        def __repr__(self):
            return "<User(name='%s', fullname='%s', password='%s')>" % (
                self.name, self.fullname, self.password)

### Configuring a Database Engine

For this tutorial we will use an in-memory-only SQLite database. This setting should live somewhere in your application. We use the __`create_engine()`__ method to create an __`Engine`__ instance, representing the core interface to the database:

In [ ]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:', echo=True)

- It has not actually tried to connect to the database yet; that happens only the first time it is asked to perform a task against the database.

__`echo`__

- This is shortcut to setting up SQLAlchemy logging, which is accomplished via Python’s standard __`logging`__ module. 
- If this is set to __`True`__, we’ll see all the generated SQL produced. _(You can set it to __`False`__. )_

### Creating the Schema

![Intermediate_3](./images/img_meta_data.png)

We call the __`MetaData.create_all()`__ method, passing in our `Engine` as a source of database connectivity. Special commands are first emitted to check for the presence of the users table then the actual __`CREATE TABLE`__ statement. Your application doesn't need to run this all the time. This is only executed to create the required tables so this should be in a separate script.

In [ ]:
Base.metadata.create_all(engine)

### Creating a Session

We’re now ready to start talking to the database. In SQLAlchemy, the ORM’s “handle” to the database is the __`Session`__.

When we first set up the application, at the same level as our __`create_engine()`__ statement, we define a __`Session`__ class which will serve as a factory for new __`Session`__ objects. This custom-made __`Session`__ class will create new __`Session`__ objects which are bound to our database:

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)

session = Session()

The above __`session`__ is associated with our SQLite-enabled Engine, but it hasn’t opened any connections yet. The session is defined one time similar to how your database is defined only once in your application.

When it’s first used, it retrieves a connection from a pool of connections maintained by the Engine, and holds onto it until we commit all changes and/or close the session object. Handling the session depends on how you structure and configure your application.

### Creating an Instance of the Mapped Class

Let’s create and inspect a __`User`__ object. In your controllers or __`views.py`__ creating objects will look like this:

In [ ]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')

In [ ]:
ed_user.name

In [ ]:
ed_user.password

In [ ]:
str(ed_user.id)

Even though we didn’t specify it in the constructor, the id attribute still produces a value of __`None`__ when we access it.

### Adding and Updating Objects in a Session

To persist our __`User`__ object, we `add()` it to our __`Session`__:

In [ ]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')

session.add(ed_user)

At this point, we say that the instance is pending; no SQL has yet been issued and the object is not yet represented by a row in the database. The Session will issue the SQL to persist Ed Jones as soon as is needed, using a process known as a flush. If we query the database for Ed Jones, all pending information will first be flushed, and the query is issued immediately thereafter.

For example, below we create a new Query object which loads instances of User. We “filter by” the name attribute of ed, and indicate that we’d like only the first result in the full list of rows. A User instance is returned which is equivalent to that which we’ve added:

In [ ]:
our_user = session.query(User).filter_by(name='ed').first() 

In fact, the Session has identified that the row returned is the same row as one already represented within its internal map of objects, so we actually got back the identical instance as that which we just added:

In [ ]:
ed_user is our_user

The ORM concept at work here is known as an identity map and ensures that all operations upon a particular row within a Session operate upon the same set of data. Once an object with a particular primary key is present in the Session, all SQL queries on that Session will always return the same Python object for that particular primary key; it also will raise an error if an attempt is made to place a second, already-persisted object with the same primary key within the session.

We can add more User objects at once using __`add_all()`__:

In [ ]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', password='foobar'),
    User(name='mary', fullname='Mary Contrary', password='xxg527'),
    User(name='fred', fullname='Fred Flinstone', password='blah')])

Also, we’ve decided the password for Ed isn’t too secure, so lets change it:

In [ ]:
ed_user.password = 'f8s7ccs'

The Session is paying attention. It knows, for example, that Ed Jones has been modified and that three new User objects are pending::

In [ ]:
session.dirty

In [ ]:
session.new

We tell the Session that we’d like to issue all remaining changes to the database and commit the transaction, which has been in progress throughout. We do this via commit(). The Session emits the UPDATE statement for the password change on “ed”, as well as INSERT statements for the three new User objects we’ve added:

In [ ]:
session.commit()

__`commit()`__ flushes whatever remaining changes remain to the database, and commits the transaction. The connection resources referenced by the session are now returned to the connection pool. Subsequent operations with this session will occur in a new transaction, which will again re-acquire connection resources when first needed.

If we look at Ed’s id attribute, which earlier was None, it now has a value:

In [ ]:
ed_user.id 

After the __`Session`__ inserts new rows in the database, all newly generated identifiers and database-generated defaults become available on the instance, either immediately or via load-on-first-access. 

In this case, the entire row was re-loaded on access because a new transaction was begun after we issued __`commit()`__. SQLAlchemy by default refreshes data from a previous transaction the first time it’s accessed within a new transaction, so that the most recent state is available.

### Rolling Back Changes

Since the Session works within a transaction, we can roll back changes made too. Let’s make two changes that we’ll revert; ed_user‘s user name gets set to Edwardo:

In [ ]:
ed_user.name = 'Edwardo'

and we’ll add another erroneous user, `fake_user`:

In [ ]:
fake_user = User(name='fakeuser', fullname='Invalid', password='12345')
session.add(fake_user)

Querying the session, we can see that they’re flushed into the current transaction:

In [ ]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

Rolling back, we can see that ed_user‘s name is back to __`ed_user`__, and __`fake_user`__ has been kicked out of the session:

In [ ]:
session.rollback()

In [ ]:
ed_user.name

In [ ]:
fake_user in session

Issuing a `SELECT` illustrates the changes made to the database:

In [ ]:
session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()

### Querying the Session

A Query object is created using the __`query()`__ method on __`Session`__. This function takes a variable number of arguments, which can be any combination of classes and class-instrumented descriptors. 

Below, we indicate a Query which loads User instances. When evaluated in an iterative context, the list of User objects present is returned:

In [ ]:
for instance in session.query(User).order_by(User.id):
        print(instance.name, instance.fullname)

You can control the names of individual column expressions using the __`label()`__ construct, which is available from any ColumnElement-derived object, as well as any class attribute which is mapped to one (such as `User.name`):

In [ ]:
for row in session.query(User.name.label('name_label')).all():
    print(row.name_label)

The name given to a full entity such as `User`, assuming that multiple entities are present in the call to __`query()`__, can be controlled using __`aliased()`__:

In [ ]:
from sqlalchemy.orm import aliased
user_alias = aliased(User, name='user_alias')

for row in session.query(user_alias, user_alias.name).all():
    print(row.user_alias)

Basic operations with Query include issuing __`LIMIT`__ and __`OFFSET`__, most conveniently using Python array slices and typically in conjunction with __`ORDER BY`__:

In [ ]:
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

And filtering results, which is accomplished either with `filter_by()`, which uses keyword arguments:

In [ ]:
for name, in session.query(User.name).filter_by(fullname='Ed Jones'):
    print(name)

Or __`filter()`__, which uses more flexible SQL expression language constructs. These allow you to use regular Python operators with the class-level attributes on your mapped class:

In [ ]:
for name, in session.query(User.name).filter(User.fullname=='Ed Jones'):
    print(name)

### Summary

ORM's are huge concepts to tackle. These ORM's usually consist of big libraries or are frameworks themselves. They abstract away the database layer, enabling you to swap out databases at will and yet have uniform code to query the database. Querying the database using an ORM can be done in many ways as shown in th examples.

Our SQLAlchemy code may be organized and summed up like this:

In [ ]:
# In your settings file
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:', echo=True)


# In your models file
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class User(Base):
        __tablename__ = 'users'

        id = Column(Integer, primary_key=True)
        name = Column(String)
        fullname = Column(String)
        password = Column(String)

        def __repr__(self):
            return "<User(name='%s', fullname='%s', password='%s')>" % (
                self.name, self.fullname, self.password)


# In your database creation script
Base.metadata.create_all(engine)


# In your session handler
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

# When creating in any controller
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')
session.add(ed_user)

session.commit() or session.rollback()

# When querying in any controller
session.query(User.name).filter_by(fullname='Ed Jones')

## 18.2 Peewee

> _Peewee is a simple and small ORM. It has few (but expressive) concepts, making it easy to learn and intuitive to use._

Let's install Peewee.

In [ ]:
!pip install peewee

### Model Definition

When starting a project with Peewee, it’s typically best to begin with your data model, by defining one or more `Model` classes. We'll use an in-memory SQLite database:

In [ ]:
import peewee

db = peewee.SqliteDatabase(':memory:')


class BaseModel(peewee.Model):

    class Meta:
        database = db


class Person(BaseModel):
    name = peewee.CharField()
    birthday = peewee.DateField()
    is_relative = peewee.BooleanField()

Now that we have our model, let’s connect to the database. Although it’s not necessary to open the connection explicitly, it is good practice since it will reveal any errors with your database connection immediately, as opposed to some arbitrary time later when the first query is executed. It is also good to close the connection when you are done – for instance, a web app might open a connection when it receives a request, and close the connection when it sends the response.

### Creating the Database

We’ll begin by creating the tables in the database that will store our data. This will create the tables with the appropriate columns, indexes, sequences, and foreign key constraints:

In [ ]:
models = [
    Person,
]
db.connect()
db.create_tables(models)

### Storing data

Let’s begin by populating the database with some people. We will use the __`save()`__ and __`create()`__ methods to add and update people’s records.

In [ ]:
from datetime import date

# You can add a person by calling the save() method on an instance:
uncle_bob = Person(name='Bob', birthday=date(1960, 1, 15), is_relative=True)
uncle_bob.save()

# You can also add a person by calling the create() method, which returns an instance:
grandma = Person.create(name='Grandma', birthday=date(1935, 3, 1), is_relative=True)
herb = Person.create(name='Herb', birthday=date(1950, 5, 5), is_relative=False)

### Updating Data

Updating is as simple as editing and the calling __`save()`__ on the instance:

In [ ]:
grandma.name = 'Grandma L.'
grandma.save()  # Update grandma's name in the database.

### Removing Data

Removing data is as simple as calling the `delete_instance()` method:

In [ ]:
herb.delete_instance()

### Retrieving Data

The real strength of our database is in how it allows us to retrieve data through queries. Relational databases are excellent for making ad-hoc queries.

#### Getting single records

Let’s retrieve Grandma’s record from the database. To get a single record from the database, use `SelectQuery.get()` or the equivalent shorthand `Model.get()`:

In [ ]:
grandma = Person.select().where(Person.name=='Grandma L.').get()

# is equivalent to

grandma = Person.get(Person.name=='Grandma L.')